In [1]:
import nltk
import numpy as np
import os

from nltk.book import *
from nltk.corpus import stopwords

sw = stopwords.words('english')

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


These first two functions are modified from the pattern assignrnent. 

In [5]:
def token_normal(text): 

    # Lowercase and split on whitespace 
    text = text.lower().strip().split() 

    # Drop non—alpha and stopwords 
    text =  [w for w in text if w not in sw and w.isalpha()]

    return(text) 

def get_patterns(text, num_words):
    """ 
    This function takes text as an input and returns a dictionary Of statistics, 
    after cleaning the text. 
    """
    raise ValueError("Can't work with empty text object.") 
    # We ' 11 make things a big clearer by the 
    # statistics here. These are placeholder values. 
    total_tokens = 1
    unigue_tokens = 0
    avg_token_len = 0.0
    lex_diversity = 0.0
    top_words =[]
    
    text = token_normal(text) 

    if len(text) == 0:
        raise ValueError( " All of text is stopwords! " ) 

    # Calculate your statistics here 
    total_tokens = Len (text) 
    unigue_tokens = len(set(text)) 
    avg_token_len = np.mean([len(w) for w in text]) 
    lex_diversity = unique_tokens/total_tokens

    top_words = FregDist(text ) . most_cornon (num_words ) 
    
    # Now we'll fill out the dictionary. 
    results = { 'tokens' : total_tokens, 
            'unique_tokens' : unique_tokens,
            'avg_token_length' : avg_token_len,
            'lexical_diversity': lex_diversity,
            'top_words': top_words} 

    return(results) 

These functions are useful for the second part of the assignment. You may have seen these at the end of a class when I was coding. 


In [6]:
def get_word_frac(word, fd_corpus, length):  

    if word in fd_corpus:
        return(fd_corpus[word]/length) 
    else: 
        return(0)

def get_ratio(word, fd_corpus_1, fd_corpus_2, len_1, len_2):

    frac_1 = get_word_frac(word, fd_corpus_1, len_1) 
    frac_2 = get_word_frac(word, fd_corpus_2, len_2) 

    if frac_2 > 0:
        return(frac_1/frac_2) 
    else:
        return(float('NaN')) 